### Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/03/13 15:32:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 15:32:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/13 15:32:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Configurações Pandas

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [3]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/banco_original_enriched')
           )

### Location

In [4]:
df_location = (df_input
               .select('codmunres_sinasc').distinct()
               .withColumn('city', F.lit(None))
               .withColumn('state', F.substring('codmunres_sinasc', 1,2))
               .withColumn('county', F.col('codmunres_sinasc'))
               .withColumn('address_1', F.lit(None))
               .withColumn('address_2', F.lit(None))
               .withColumn('zip', F.lit(None))
               .withColumn('location_source_value', F.lit(None))
               .withColumn('country_concept_id', F.lit(None))
               .withColumn('country_source_value', F.lit(None))
               .withColumn('latitude', F.lit(None))
               .withColumn('longitude', F.lit(None))
               .withColumn('location_id', F.col('codmunres_sinasc'))
               .distinct()
            ).select(
                    F.col('location_id').cast('integer'),
                    F.col('city').cast('string'),
                    F.col('state').cast('string'),
                    F.col('county').cast('string'),
                    F.col('address_1').cast('string'),
                    F.col('address_2').cast('string'),
                    F.col('zip').cast('string'),
                    F.col('location_source_value').cast('string'),
                    F.col('country_concept_id').cast('integer'),
                    F.col('country_source_value').cast('string'),
                    F.col('latitude').cast('float'),
                    F.col('longitude').cast('float')
                    )

df_location.count()

11158

## Salvando no Postgresql

In [5]:
write_to_postgresl(df_location, tb_name='location', write_mode='append')

'Total time: 0:00:03.798135 and - Total rows: 11158 - Total columns: 12'

### Deprecated

### Salvando CSV 

In [ ]:
# df_location.repartition(1).write.option("quoteAll",True).csv('/data/IDAF/SHARED/omop_tables_revisadas/location', header=True, mode='overwrite')

## SQL de insertion

In [ ]:
# CREATE TABLE public.location (
# 			location_id integer NOT NULL,
# 			address_1 varchar(50) NULL,
# 			address_2 varchar(50) NULL,
# 			city varchar(50) NULL,
# 			state varchar(2) NULL,
# 			zip varchar(9) NULL,
# 			county varchar(20) NULL,
# 			location_source_value varchar(50) NULL,
# 			country_concept_id integer NULL,
# 			country_source_value varchar(80) NULL,
# 			latitude NUMERIC NULL,
# 			longitude NUMERIC NULL );


# CREATE TABLE public.location_pyspark (
# 			location_id varchar,
# 			address_1 varchar,
# 			address_2 varchar,
# 			city varchar,
# 			state varchar,
# 			zip varchar,
# 			county varchar,
# 			location_source_value varchar,
# 			country_concept_id varchar,
# 			country_source_value varchar,
# 			latitude varchar,
# 			longitude varchar);


# insert into public.location (
# location_id ,
# address_1 ,
# address_2 ,
# city ,
# state ,
# zip ,
# county ,
# location_source_value ,
# country_concept_id ,
# country_source_value ,
# latitude ,
# longitude 
# )
# SELECT 
# cast(location_id as integer) ,
# address_1 ,
# address_2 ,
# city ,
# state ,
# zip ,
# county ,
# location_source_value ,
# cast(case when country_concept_id='' then null else country_concept_id end as integer) ,
# country_source_value ,
# cast(case when latitude='' then null else latitude end as numeric) ,
# cast(case when longitude='' then null else longitude end as numeric) 
# 	FROM public.location_pyspark;
